# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [1]:
# Your code here
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [2]:
# Your code here
formula = 'len ~ C(supp) + dose'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [ ]:
# Your comment here
#Based on the interpretation from last time, lower PR(>F) values indicate more influence.
#They are both below an alpha value of 0.05, so both significant, but dose is more significant.

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [9]:
# Your code here
dfOJ = df[df.supp == 'OJ']
OJlen = dfOJ.len
dfVC = df[df.supp == 'VC']
VClen = dfVC.len


Now run a t-test between these two groups and print the associated two-sided p-value: 

In [10]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
from scipy import stats
stats.ttest_ind(OJlen, VClen)
#so this p value is not statistically significant

Ttest_indResult(statistic=1.91526826869527, pvalue=0.06039337122412849)

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [11]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)
#wow!  it's true!

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [13]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [27]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    
    for group2 in df.groupby(['supp', 'dose'])['len']:            
        group_name2 = group2[0]
        if group_name2 == group_name:
            continue
        data2 = group2[1]
    
        p=stats.ttest_ind(data, data2).pvalue
        statsig='Not statistically significant' if p>=0.05 else "Statistically Significant"
        print(group_name, 'vs', group_name2, '-- p-value:', p ,
              'compared to alpha = 0.05:', statsig
             )
    
    print()
    
    

('OJ', 0.5) vs ('OJ', 1.0) -- p-value: 8.357559281443774e-05 compared to alpha = 0.05: Statistically Significant
('OJ', 0.5) vs ('OJ', 2.0) -- p-value: 3.4018585295016214e-07 compared to alpha = 0.05: Statistically Significant
('OJ', 0.5) vs ('VC', 0.5) -- p-value: 0.005303661339923052 compared to alpha = 0.05: Statistically Significant
('OJ', 0.5) vs ('VC', 1.0) -- p-value: 0.04223992429368205 compared to alpha = 0.05: Statistically Significant
('OJ', 0.5) vs ('VC', 2.0) -- p-value: 7.025409196997986e-06 compared to alpha = 0.05: Statistically Significant

('OJ', 1.0) vs ('OJ', 0.5) -- p-value: 8.357559281443774e-05 compared to alpha = 0.05: Statistically Significant
('OJ', 1.0) vs ('OJ', 2.0) -- p-value: 0.03736279585664383 compared to alpha = 0.05: Statistically Significant
('OJ', 1.0) vs ('VC', 0.5) -- p-value: 1.3372624230559434e-08 compared to alpha = 0.05: Statistically Significant
('OJ', 1.0) vs ('VC', 1.0) -- p-value: 0.0007807261651774468 compared to alpha = 0.05: Statistical

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.